# Mapeo campos 

|Fuente | Sources |
|---|---|
|Nombre base para flora de Bogota | Taxa |
|Comentario de validación | Taxa.Comment |
|Para revisión | Taxa.CheckPriority 0 or 1 |
|Endemica | Taxa.Distribution |
|Estado de amenaza | RedListAssessments with Standard='JBB'
|Origen y status | Origins |
|Trabajo de No nativas Flora de Bogotá | Origins.Type???
|Determinador en fuente | Identifications.IdentifiedBy Merge with next field |
|Determinador Depurado | Identifications.IdentifiedBy |
|Calificador en fuente | Identifications.Certainty |
|Fecha_Determinacion en fuente | Identifications.Date |
|Fecha_Determinacion_(Year) | Identifications.Date |
|Fecha_Determinacion_(Month) | Identifications.Date |
|Fecha_Determinacion_(Day) | Identifications.Date |
|State | Areas |
|County | Areas |
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |
|Origen para herbario | Specimens.Institution  |
|Número de identificación | Specimens.SpecimenCode |
|Fecha_Inicial | REMOVE |
|Notas de altura | REMOVE ???????
|Familia Revisada | REMOVE |
|Género revisado | REMOVE |
|Comentarios de determinación en fuente | REMOVE |
|Nivel taxonómico | REMOVE, info retrieved from TPRS |
|Familia en fuente | REMOVE  |
|Género en fuente | REMOVE  |
|Epíteto específico en fuente | REMOVE  |
|Autor del epíteto específico en fuente | REMOVE  |
|Epíteto infraespecífico | REMOVE  |
|Autor del infraespecífico en fuente | REMOVE  |
|Nombre científico en fuente | REMOVE  |
|Nombre validado | REMOVE  |
|Nivel taxonómico en fuente | REMOVE  |
|Herbario de proveniencia | REMOVE  |

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import requests
import json
import re
import time

import pykew.powo as powo
from pygbif import species, registry

import credentials

In [9]:
tnrs_url = 'https://tnrsapi.xyz/tnrs_api.php'

In [118]:
def tropicos(name, api_key):
	service = 'http://services.tropicos.org/name'
	r = requests.get(
		f'{service}/search/?apikey={api_key}&name={name}&format=json'
	)
	#s = requests.get(
	#	f'{service}/{r['NameId']}/highertaxa/?apikey={api_key}&name={name}&format=json'
	#)
	
	return r.json()


In [72]:
def powo_query(name):
	act_name = ''
	bits = re.split(r'\s+', name)

	if bits[0] == '×' or bits[1] == '×':
		act_name = ' '.join(bits[:3])
	else:
		act_name = ' '.join(bits[:2])
	
	r = powo.search(act_name)

	if r.size() == 1:
		results = [i for i in r]
		return (results[0]['name'], results[0]['author'], results[0]['rank'])
	
	elif r.size() > 1:
		return f'Multiples results with name {name}'
	
	else:
		return f'No results with name {name}'

In [3]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'
ori = pd.read_csv(orifile, low_memory=False)

### Si la consulta a TNRS ya fue realizada y los resultados guardados:

In [73]:
# Map between original name and corrected version by TNRS
taxa_map_file = 'tnrs_query.csv'
taxa_map = pd.read_csv(taxa_map_file)

### De lo contrario, aqui se realiza la consulta

In [ ]:
orinames = ori['Nombre base para flora de Bogota'].unique()
query = [[i, j] for i,j in enumerate(orinames) if pd.notna(j)]

r = requests.post(
	tnrs_url, 
	headers={"Content-Type": "application/json"},
	data=json.dumps({
		'data': query,
		'opts': {
			'sources': ['wcvp','wfo'],
			'class': 'wcvp',
			'mode': 'resolve',
			'matches' : 'best'
		}		
	})	
)

In [ ]:
taxa_map = pd.DataFrame.from_records(results)
taxa_map.to_csv('tnrs_query.csv', index=False)

### Aqui se continua la normalizacion de nombres

In [ ]:
taxa_map.loc[taxa_map.Overall_score == '', 'Overall_score'] = 'NAN'
taxa_map['Overall_score'] = taxa_map.Overall_score.astype(np.float32)

### Consulta a POWO de hibridos

Primero remover hibridos no publicados, solo se registran a nivel de determinacion

In [78]:
taxa_map = taxa_map.drop(
	index=taxa_map.loc[taxa_map.Name_submitted.str.contains(r'Salvia [\w\.\s]+ ×')].index
)

In [81]:
for row in taxa_map.loc[taxa_map.Name_submitted.str.contains('×')].itertuples():
	#print(row.Name_submitted)
	po = powo_query(row.Name_submitted)
	if isinstance(po, tuple):
		taxa_map.loc[row.Index, 'Name_matched'] = po[0]
		taxa_map.loc[row.Index, 'Author_matched'] = po[1]
		taxa_map.loc[row.Index, 'Name_matched_rank'] = po[2].lower()
	else:
		print(po)
	time.sleep(1)

No results with name Glandularia × hortulanus
Multiples results with name Pelargonium × hortorum L.H.Bailey
Multiples results with name Abelia × grandiflora (Andre) Rehder
No results with name Polygala × dalmasiana
Multiples results with name Lavatera × clementii Cheek


In [101]:
# Manually correct names couldn't be matched easily by POWO,
# but there is a simple solution there 

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Pelargonium × hortorum')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Pelargonium × hortorum', 'L.H.Bailey', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Abelia × grandiflora')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Abelia × grandiflora', '(Rovelli ex André) Rehder', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Lavatera × clementii')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Lavatera × clementii', 'Cheek', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Polygala × dalmasiana')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Polygala', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Epidendrum chicoense')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Epidendrum', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Plantae')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = (np.nan, np.nan, np.nan)

# TNRS couldn't get appropriate response for the following

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Callianthe striata')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Callianthe striata', '(G.F.Dicks. ex Lindl.) Donnell', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Phlegmariurus acifolius')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Phlegmariurus acifolius', '(Rolleri) B.Øllg.', 'species')


In [104]:
# Save eliminated binomials from the database, they probably are invalid or unpublished names
taxa_map.loc[(taxa_map.Overall_score < 0.9) & (taxa_map.Name_matched_rank == 'genus'),
	['Name_submitted', 'Name_matched', 'Name_matched_rank']
	].to_csv('binomios_eliminados.csv', index=False)

In [110]:
taxa_map[['kingdom', 'phylum', 'class', 'order', 'family']] = (np.nan, np.nan, np.nan, np.nan, np.nan)

In [ ]:
for row in taxa_map.itertuples():
	gbif = species.name_backbone(row.Name_matched, rank=row.Name_matched_rank)
	print(row.Name_matched)
	#print(json.dumps(gbif, indent=4))
	hier = [np.nan, np.nan, np.nan, np.nan, np.nan]
	if 'kingdom'in gbif:
		hier[0] = gbif['kingdom']
	if 'phylum'in gbif:
		hier[1] = gbif['phylum']
	if 'class'in gbif:
		hier[2] = gbif['class']
	if 'order'in gbif:
		hier[3] = gbif['order']
	if 'family'in gbif:
		hier[4] = gbif['family']
	taxa_map.loc[row.Index, ['kingdom', 'phylum', 'class', 'order', 'family']] = hier
	time.sleep(1)

Cirsium
Cotula australis
Munnozia senecionidis
Solanum stellatiglandulosum
Thelypteris elegantula
Ageratina vacciniaefolia
Miconia biappendiculata
Oreopanax mutisianus
Pentacalia vaccinioides
Viburnum triphyllum
Aragoa
Asteraceae
Diplostephium phylicoides
Ericaceae
Geranium
Halenia
Paepalanthus
Stevia lucida
Oxalis medicaginea
Elaphoglossum
Polypodium
Abutilon
Cestrum
Saxifragaceae
Cuphea ciliata
Althaea officinalis
Borago officinalis
Canna indica
Hypochaeris sessiliflora
Margyricarpus pinnatus
Niphogeton ternata
Origanum vulgare
Plantago linearis
Vasconcellea pubescens
Verbena officinalis
Zea mays
Abutilon hybridum
Asplenium harpeodes
Asplenium monanthes
Athyrium dombeyi
Azolla filiculoides
Bartsia pedicularoides
Blechnum cordatum
Carex pichinchensis
Cystopteris fragilis
Diphasiastrum thyoides
Diphasium jussiaei
Gaiadendron punctatum
Isoetes andina
Lycopodium clavatum
Moritzia lindenii
Pecluma eurybasis
Phlegmariurus cruentus
Selaginella
Selaginella cavifolia
Ageratina pichinchensis
A

In [ ]:
# Get hierarchy from majority rule in first 100 GBIF datasets

fields = ['kingdom', 'phylum', 'class', 'order', 'family']

for row in taxa_map.loc[taxa_map.kingdom.isna() | taxa_map.phylum.isna() | taxa_map['class'].isna() | taxa_map.order.isna() | taxa_map.family.isna()].itertuples():
	counter = {x:{} for x in fields}
	w = []
	r = species.name_lookup(row.Name_matched)
	for resu in r['results']:
		for f in fields:
			if f in resu:
				counter[f][resu[f]] = counter[f].get(resu[f], 0) + 1

	for f in fields:
		win,wcount = None, 0
		for elem in counter[f]:
			if counter[f][elem] > wcount:
				win = elem
				wcount = counter[f][elem]
		w.append(win)

	taxa_map.loc[row.Index, fields] = w

In [186]:
# Remove orphan phyla : tracheophyta, magnoliophyta, pteridophyta.

taxa_map = taxa_map.drop(
	index=taxa_map.loc[taxa_map.kingdom.isna()].index
)

In [ ]:
# Remove lichens: Ramalina, Usnea, Sticta, Candelariella

taxa_map = taxa_map.drop(
	index=taxa_map[taxa_map.Name_submitted.isin(['Ramalina', 'Usnea', 'Sticta', 'Candelariella'])].index
)

In [187]:
taxa_map.to_csv('taxa_map.csv', index=False)

### Verificación nombre realizada y guadada localmente

In [2]:
taxa_map = pd.read_csv('taxa_map.csv')

In [12]:
taxa_map.loc[
	(taxa_map.Genus_matched == 'Stelis'),
	['Name_matched', 'kingdom', 'phylum', 'class', 'order', 'family']
]

,Name_matched,kingdom,phylum,class,order,family
1551,Stelis,Plantae,Arthropoda,Liliopsida,Asparagales,Orchidaceae
1552,Stelis argentata,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1553,Stelis ascendens,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1554,Stelis asseris,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1555,Stelis calceolaris,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1556,Stelis cucullata,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1557,Stelis galeata,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1558,Stelis ligulata,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1559,Stelis parvilabris,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae
1560,Stelis pulchella,Plantae,Tracheophyta,Liliopsida,Asparagales,Orchidaceae


In [10]:
taxa_map.loc[
	taxa_map.phylum.isin(
		['Arthropoda', 'Ascomycota','Mollusca', 'Bryophyta', 'Chordata']
	),
	['Name_matched', 'kingdom', 'phylum', 'class', 'order', 'family']
]

,Name_matched,kingdom,phylum,class,order,family
1551,Stelis,Plantae,Arthropoda,Liliopsida,Asparagales,Orchidaceae
2291,Aa,Fungi,Ascomycota,Magnoliopsida,Asparagales,Orchidaceae
2368,Hebe,Plantae,Arthropoda,Magnoliopsida,Lamiales,Plantaginaceae
2420,NaN,Animalia,Arthropoda,Trilobita,Russulales,Russulaceae
2582,Modiola,Plantae,Mollusca,Bivalvia,Mytilida,Malvaceae
2900,Sematophyllaceae,Plantae,Bryophyta,Bryopsida,Hypnales,Sematophyllaceae
3166,Polylepis,Plantae,Chordata,Magnoliopsida,Rosales,Rosaceae
3342,Laestadia,Fungi,Ascomycota,Magnoliopsida,Asterales,Asteraceae


In [ ]:
conn_str = 'mysql+mysqlconnector://' + \
	f'{credentials.mysql['username']}:{credentials.mysql['password']}' + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [7]:
df = pd.read_sql(sql='SELECT * FROM Taxa', con=connection)

In [8]:
df

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp


In [9]:
connection.close()